In [136]:
import pandas as pd
import numpy as np
import math
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

# 1. Load + Clean Data
Since there were no nonsensical values found after describing each feature, we only need to filter out NA values and duplicates

In [137]:
# read data into df + describe
df = pd.read_csv("Salary Data.csv")
df.describe()
df.head()
df.info()

df['Gender'].describe() # no non-sensical genders
df['Age'].describe() # no negative ages! 
df['Years of Experience'].describe() # no negative/non-sensical years of experiences
df['Salary'].describe() # no negative salaries!
df['Education Level'].describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  373 non-null    float64
 1   Gender               373 non-null    object 
 2   Education Level      373 non-null    object 
 3   Job Title            373 non-null    object 
 4   Years of Experience  373 non-null    float64
 5   Salary               373 non-null    float64
dtypes: float64(3), object(3)
memory usage: 17.7+ KB


count            373
unique             3
top       Bachelor's
freq             224
Name: Education Level, dtype: object

In [138]:
# clean data
df = df.dropna()
df = df.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 0 to 371
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Age                  324 non-null    float64
 1   Gender               324 non-null    object 
 2   Education Level      324 non-null    object 
 3   Job Title            324 non-null    object 
 4   Years of Experience  324 non-null    float64
 5   Salary               324 non-null    float64
dtypes: float64(3), object(3)
memory usage: 17.7+ KB


# 2. Exploratory Data Analysis

1. histogram of target variable (salary)
2. pie chart: gender distribution
3. pie chart: education level
4. job title???
5. histogram: years of experience

In [ ]:
# salary histogram
salary_hist = px.histogram(df, x = "Salary", nbins = 20)
salary_hist.show()

In [ ]:
# gender pie chart
gender_count = df["Gender"].value_counts().reset_index()
gender_pie = px.pie(gender_count, values = "count", names = "Gender")
gender_pie.show()

In [141]:
# gender versus salary
gender_salary = df[['Gender', 'Salary']]
gender_salary.groupby(['Gender']).mean()
gender_salary_bar = px.bar(gender_salary, x = 'Gender', y = 'Salary')
gender_salary_bar.show()

In [ ]:
# education level pie chart
education_count = df["Education Level"].value_counts().reset_index()
education_pie = px.pie(education_count, values = "count", names = "Education Level")
education_pie.show()

# 3. Model Training + Evaluation
1. Model 1: one shot, dummy variables for all categoricals
2. Model 2: remove job titles, dummy vars for rest of categoricals

## Model 1: one shot, dummy variables for all categoricals
mean squared error: 1.2641081232274098e+29, root mean squared error: 355542982384325.75, r^2 = -5.3851067985724146e+19, 

### Training

In [ ]:
# set x + y
# make dummy variables for categorical vars
df_1 = pd.get_dummies(df)
y_1 = df_1['Salary']
X_1 = df_1.drop(['Salary'], axis = 1)

In [ ]:
## split data 70/20/10 (train/validation/test)
# split the data into train + test/validation
X_train_1, X_tv_1, y_train_1, y_tv_1 = train_test_split(X_1, y_1, train_size = 0.7, random_state = 8421)
X_train_1.shape, X_tv_1.shape, y_train_1.shape, y_tv_1.shape

# split test/validation into validation and test
X_test_1, X_valid_1, y_test_1, y_valid_1 = train_test_split(X_tv_1, y_tv_1, train_size = 0.33, random_state = 8421)
X_test_1.shape, X_valid_1.shape, y_test_1.shape, y_valid_1.shape

In [ ]:
# make model
model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)
y_pred_1 = model_1.predict(X_valid_1)

### Evaluation

In [ ]:
#mean_squared_error(y_valid_1, y_pred_1)
#model_1.score(X_test_1, y_test_1) # r^2
math.sqrt(mean_squared_error(y_valid_1, y_pred_1))

In [ ]:
coeff_df_1 = pd.DataFrame({'Feature': X_1.columns.tolist(), 'Coefficient': abs(model_1.coef_)})
coeff_df_1.sort_values("Coefficient")

## Model 2: remove job titles, dummy vars for rest of categoricals
mean squared error: 185797336.42907345, root mean squared error: 13630.749664969768, r^2 = 0.8397084956789058

### Training

In [ ]:
# make male = 0, female = 1, drop job title
df_2 = df
df_2['Gender'] = df_2['Gender'].replace("Male", 0)
df_2['Gender'] = df_2['Gender'].replace("Female", 1)
df_2 = df_2.drop(['Job Title'], axis = 1)
df_2 = pd.get_dummies(df_2)

# set x + y
y_2 = df_2['Salary']
X_2 = df_2.drop(['Salary'], axis = 1)

In [ ]:
## split data 70/20/10 (train/validation/test)
# split the data into train + test/validation
X_train_2, X_tv_2, y_train_2, y_tv_2 = train_test_split(X_2, y_2, train_size = 0.7, random_state = 8421)
X_train_2.shape, X_tv_2.shape, y_train_2.shape, y_tv_2.shape

# split test/validation into validation and test
X_test_2, X_valid_2, y_test_2, y_valid_2 = train_test_split(X_tv_2, y_tv_2, train_size = 0.33, random_state = 8421)
X_test_2.shape, X_valid_2.shape, y_test_2.shape, y_valid_2.shape

In [ ]:
# make model
model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)
y_pred_2 = model_2.predict(X_valid_2)

### Evaluation

In [ ]:
mean_squared_error(y_valid_2, y_pred_2)
#model_2.score(X_test_2, y_test_2) # r^2
math.sqrt(mean_squared_error(y_valid_2, y_pred_2))

In [ ]:
coeff_df_2 = pd.DataFrame({'Feature': X_2.columns.tolist(), 'Coefficient': abs(model_2.coef_)})
coeff_df_2.sort_values("Coefficient")